<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

### Corpus

In [1]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [3]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [4]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)
vectorizer.idf_.shape

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


(9,)

In [5]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [6]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])
skl_output.shape

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


(4, 9)

In [7]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output.toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Custom implementation

In [8]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np

In [9]:
#The code concept is taken from reference of the assignment given
def fit(inp_corpus):    
    uni_corpus = set()
    for i in inp_corpus: 
        for j in i.split():
            uni_corpus.add(j)
    uni_corpus = sorted(list(uni_corpus))
    
    corpusidf = []

    for i in uni_corpus:
        count = 0
        for j in inp_corpus:
            if i in j:
                count += 1
        count = 1 + math.log(((len(corpus)+1)/(count+1)))
        corpusidf.append(count)
    
    dic = dict(zip(uni_corpus, corpusidf))
    return dic

In [10]:
def transform(corp,x):
    corpustf = []
    
    for i in corp:
        k = i.split()
        new = []
        for j in list(x.keys()):
            count = 0
            for l in k:
                if l == j:
                    count+=1
            count = (count)/(len(k))
            new.append(count)
        corpustf.append(new)
    
    
    tfidf = [] 
    values = list(x.values())

    c = 1
    for i in range(len(corpustf)):
        new = []
        for j in range(len(values)):
            c = corpustf[i][j]*values[j]
            new.append(c)
        tfidf.append(new)
    
    tfidff = normalize(tfidf)
    
    spa_tfidf = csr_matrix(tfidff)
    
    return(spa_tfidf)
    

In [11]:
x = fit(corpus)
tfidf = transform(corpus,x).toarray()

In [12]:
print(list(x.keys()))

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [13]:
print(tfidf)

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> Modifying fit and transform functions so that vocab will contain only 50 terms with top idf scores.</li
        
</ul>

In [ ]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open(r'C:\Users\Admin\Desktop\Applied projects\Assignment\5. Implementing tf-idf vectorizer\cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

print(corpus)

In [15]:
from scipy.sparse import csr_matrix
import math
from sklearn.preprocessing import normalize
import numpy as np

In [16]:
def fit(inp_corpus):    
    uni_corpus = set()
    for i in inp_corpus: 
        for j in i.split():
            uni_corpus.add(j)
    uni_corpus = sorted(list(uni_corpus))
    
    corpusidf = []

    for i in uni_corpus:
        count = 0
        for j in inp_corpus:
            if i in j:
                count += 1
        count = 1 + math.log(((len(corpus)+1)/(count+1)))
        corpusidf.append(count)
    
    y = dict(zip(uni_corpus, corpusidf))
    
    a_list = list(y.values())
    b_list = list(y.keys())
    a = []
    b = []

    for num in range(len(a_list)):
        if num<50:
            a.append(a_list[num])
        
    for num in range(len(b_list)):
        if num<50:
            b.append(b_list[num])

    dic = dict(zip(b,a))
   

    return dic

In [17]:
def transform(corp,x):
    corpustf = []
    
    for i in corp:
        k = i.split()
        new = []
        for j in list(x.keys()):
            count = 0
            for l in k:
                if l == j:
                    count+=1
            count = (count)/(len(k))
            new.append(count)
        corpustf.append(new)
    
    
    tfidf = [] 
    values = list(x.values())

    c = 1
    for i in range(len(corpustf)):
        new = []
        for j in range(len(values)):
            c = corpustf[i][j]*values[j]
            new.append(c)
        tfidf.append(new)
    
    tfidff = normalize(tfidf)
    
    spa_tfidf = csr_matrix(tfidff)
    
    return(spa_tfidf)

In [18]:
x = fit(corpus)
#tfidf = transform(corpus,x).toarray()

In [19]:
y = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse = True)}

In [20]:
print(y)

{'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.922918004572872, 'abstruse': 6.922918004572872, 'academy': 6.922918004572872, 'accents': 6.922918004572872, 'accessible': 6.922918004572872, 'acclaimed': 6.922918004572872, 'accolades': 6.922918004572872, 'accurately': 6.922918004572872, 'achille': 6.922918004572872, 'ackerman': 6.922918004572872, 'adams': 6.922918004572872, 'added': 6.922918004572872, 'admins': 6.922918004572872, 'admiration': 6.922918004572872, 'admitted': 6.922918004572872, 'adrift': 6.922918004572872, 'adventure': 6.922918004572872, 'aesthetically': 6.922918004572872, 'affected': 6.922918004572872, 'affleck': 6.922918004572872, 'afternoon': 6.922918004572872, 'abysmal': 6.517452896464707, 'accurate': 6.517452896464707, 'accused': 6.517452896464707, 'achievement': 6.517452896464707, 'adaptation': 6.517452896464707, 'advise': 6.517452896464707, 'afraid': 6.517452896464707, 'africa': 6.517452896464707, 'ability': 6.229770824012927, 'acted': 6.

In [31]:
tfidf = transform(corpus,x).toarray()
tfidf.shape

(746, 50)